In [1]:
import numpy as np
from localizer import detect_objects

detect_objects(np.ones((255, 255, 3)))

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
YOLOv5 🚀 2022-7-29 Python-3.10.5 torch-1.11.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


detections Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []


(array([[[          1,           1,           1],
         [          1,           1,           1],
         [          1,           1,           1],
         ...,
         [          1,           1,           1],
         [          1,           1,           1],
         [          1,           1,           1]],
 
        [[          1,           1,           1],
         [          1,           1,           1],
         [          1,           1,           1],
         ...,
         [          1,           1,           1],
         [          1,           1,           1],
         [          1,           1,           1]],
 
        [[          1,           1,           1],
         [          1,           1,           1],
         [          1,           1,           1],
         ...,
         [          1,           1,           1],
         [          1,           1,           1],
         [          1,           1,           1]],
 
        ...,
 
        [[          1,           1